# Exercise 6: Quantum Fourier Transform

QFT Unitary
$$F_N=\frac{1}{\sqrt{N}}\begin{bmatrix}
1&1&1&1&\cdots &1 \\
1&\omega_n&\omega_n^2&\omega_n^3&\cdots&\omega_n^{N-1} \\
1&\omega_n^2&\omega_n^4&\omega_n^6&\cdots&\omega_n^{2(N-1)}\\ 1&\omega_n^3&\omega_n^6&\omega_n^9&\cdots&\omega_n^{3(N-1)}\\
\vdots&\vdots&\vdots&\vdots&&\vdots\\
1&\omega_n^{N-1}&\omega_n^{2(N-1)}&\omega_n^{3(N-1)}&\cdots&\omega_n^{(N-1)(N-1)}
\end{bmatrix}
$$

where $$\omega_n= e^{\frac{2 \pi i}{N}}$$ 

for example, for 2 qubits, the operator is:

$$F_4=\frac{1}{2}\begin{bmatrix}
1&1&1&1 \\
1&e^{\frac{\pi i}{2}}&e^{\pi i}&e^{\frac{3 \pi i}{2}} \\
1&e^{\pi i}&e^{2 \pi i}&e^{\frac{6 \pi i}{2}}\\
1&e^{\frac{3 \pi i}{2}}&e^{\frac{6 \pi i}{2}}&e^{\frac{9 \pi i}{2}}
\end{bmatrix}
=\frac{1}{2}\begin{bmatrix}
1&1&1&1 \\
1&i&-1&-i \\
1&-1&1&-1\\
1&-i&-1&-i
\end{bmatrix}
$$

So, appying $F_4$ on the state $|\phi>=\frac{1}{2}[1,-1,1,-1]^T=\frac{1}{2}(|00>-|01>+|10>-|11>)$

$$F_4|\phi>=\frac{1}{2}\begin{bmatrix}
1&1&1&1 \\
1&i&-1&-i \\
1&-1&1&-1\\
1&-i&-1&-i
\end{bmatrix}\frac{1}{2}\begin{bmatrix}1\\-1\\1\\-1\end{bmatrix}=\frac{1}{4}\begin{bmatrix}0\\0\\4\\0\end{bmatrix}=\begin{bmatrix}0\\0\\1\\0\end{bmatrix}=|10>=|2>=|N/r>$$

being $N=2^{numberQubits}$ and $r$ the period of the function (in this case, 2)



[Example](../quirk.html#circuit=%7B%22cols%22%3A%5B%5B%22Y%22%5D%2C%5B%22H%22%2C%22H%22%2C%22H%22%2C%22H%22%2C%22H%22%2C%22H%22%5D%2C%5B%22QFT6%22%5D%5D%7D)

ProjectQ implements a single gate for QFT, but in the current version, it needs at the end, additional permutations


In [1]:
import projectq
from projectq.backends import CommandPrinter
from projectq.ops import QFT, All, Measure, Y, H, Swap
from projectq.backends import Simulator
from projectq.setups.default import get_engine_list
def get_state_as_str(eng,qubits,cheat=False):
    import numpy as np
    s=""
    if (cheat):
        print("Cheat: ", eng.backend.cheat())
    if (len(qubits)==1):
        for i in range(2):
            #print("bits:%d%s"%(i,bits))
            a=eng.backend.get_amplitude("%d"%(i),qubits)
            if (a.real!=0)|(a.imag!=0):
                if s!="":
                    s=s+"+"
                a="({:.2f})".format(a)
                s=s+"%s|%d>"%(a,i)

    else:
        for j in range(2**(len(qubits)-1)):
            bits=np.binary_repr(j,width=len(qubits)-1)
            #print("Bits:",j,bits)
            for i in range(2):
                #print("bits:%d%s"%(i,bits))
                a=eng.backend.get_amplitude("%d%s"%(i,bits[-1::-1]),qubits)
                if (a.real!=0)|(a.imag!=0):
                    if s!="":
                        s=s+"+"
                    a="({:.2f})".format(a)
                    s=s+"%s|%s%d>"%(a,bits,i)
                #print(s)
    return(s)



In [2]:
# Use the default compiler engines with a CommandPrinter in the end:
engines2 = get_engine_list() # + [CommandPrinter()]

eng = projectq.MainEngine(backend=Simulator(), engine_list=engines2)

#Select the number of qubits
q = eng.allocate_qureg(6)


Y | q[0]
All(H) | q
eng.flush()
print("Initial state:",get_state_as_str(eng,q))

QFT | q
for i in range(len(q)//2):
        Swap | (q[i],q[len(q)-i-1])
eng.flush()

print("\nQFT:",get_state_as_str(eng,q))

All(Measure) | q

del eng

Initial state: (0.00+0.12j)|000000>+(0.00-0.12j)|000001>+(0.00+0.12j)|000010>+(0.00-0.12j)|000011>+(0.00+0.12j)|000100>+(0.00-0.12j)|000101>+(0.00+0.12j)|000110>+(0.00-0.12j)|000111>+(0.00+0.12j)|001000>+(0.00-0.12j)|001001>+(0.00+0.12j)|001010>+(0.00-0.12j)|001011>+(0.00+0.12j)|001100>+(0.00-0.12j)|001101>+(0.00+0.12j)|001110>+(0.00-0.12j)|001111>+(0.00+0.12j)|010000>+(0.00-0.12j)|010001>+(0.00+0.12j)|010010>+(0.00-0.12j)|010011>+(0.00+0.12j)|010100>+(0.00-0.12j)|010101>+(0.00+0.12j)|010110>+(0.00-0.12j)|010111>+(0.00+0.12j)|011000>+(0.00-0.12j)|011001>+(0.00+0.12j)|011010>+(0.00-0.12j)|011011>+(0.00+0.12j)|011100>+(0.00-0.12j)|011101>+(0.00+0.12j)|011110>+(0.00-0.12j)|011111>+(0.00+0.12j)|100000>+(0.00-0.12j)|100001>+(0.00+0.12j)|100010>+(0.00-0.12j)|100011>+(0.00+0.12j)|100100>+(0.00-0.12j)|100101>+(0.00+0.12j)|100110>+(0.00-0.12j)|100111>+(0.00+0.12j)|101000>+(0.00-0.12j)|101001>+(0.00+0.12j)|101010>+(0.00-0.12j)|101011>+(0.00+0.12j)|101100>+(0.00-0.12j)|101101>+(0.00+0.12j)|101110